In [ ]:
import pandas as pd
import os
from tqdm.autonotebook import tqdm

In [ ]:
df = pd.read_json("/home/jupyter/datasets/microbioevents/tokenized/microbioevents.json", orient = "records", dtype="object")

In [ ]:
df.head()

In [ ]:
#drop records with no HADM_ID
df = df[df['HADM_ID']!='NaN']

In [ ]:
df.info(verbose = True, null_counts=True)

In [ ]:
df_ICU = pd.read_csv("/home/jupyter/datasets/raw/ICUSTAYS.csv", dtype = "object")

In [ ]:
df_ICU.head()

In [ ]:
df_ICU.info(verbose = True, null_counts=True)

In [ ]:
# Selecting list of HADM_ID with single ICUSTAY
df_single_ICU = df_ICU.groupby(by=['HADM_ID'], as_index=False).agg({'ICUSTAY_ID': 'count'})
df_single_ICU  = df_single_ICU[df_single_ICU['ICUSTAY_ID']==1]['HADM_ID']

In [ ]:
#number of HADM_ID with single ICUSTAY
len(df_single_ICU)

In [ ]:
# Dropping HADM_ID with mutiple ICUSTAYS
df = df[df['HADM_ID'].isin(df_single_ICU)].reset_index(drop = True)

In [ ]:
df.info(verbose = True, null_counts=True)

In [ ]:
df = df.merge(df_ICU[['HADM_ID','INTIME']], how='left',left_on='HADM_ID', right_on='HADM_ID')

In [ ]:
df = df.rename(columns={'INTIME':'ICU_INTIME'})

In [ ]:
df.head()

In [ ]:
df = df.fillna('NaN')

In [ ]:
df = df.rename(columns={'CHARTTIME':'STORETIME'})

In [ ]:
df['STORETIME'] = pd.to_datetime(df['STORETIME'], errors='coerce')
df['ICU_INTIME'] = pd.to_datetime(df['ICU_INTIME'], errors='coerce')

In [ ]:
%%time
df['icu_tdelta'] = df['STORETIME'] - df['ICU_INTIME']
df['icu_tdelta'] = df['icu_tdelta'].map(lambda x: x.total_seconds())

In [ ]:
df['STORETIME'] = df['STORETIME'].map(lambda x: str(x))

In [ ]:
df.head()

In [ ]:
sec_24hrs = 24*60*60 #total seconds in 24 hrs
sec_48hrs = 48*60*60 #total seconds in 48 hrs

In [ ]:
print("Total Records: " + str(len(df)))
print("Total Records Prior to 24hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_24hrs])))
print("Total Records Prior to 48hrs in ICU: " +str(len(df[df['icu_tdelta']<sec_48hrs])))

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets//data_before_24hrs_icu/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# extract data recorded in ans prior to first 2hhrs of ICU stay
df[df['icu_tdelta']<sec_24hrs].to_json(mypath_output+"microbioevents.json", orient = 'records')

In [ ]:
# create output path
#mypath_output = "/home/jupyter/datasets/data_before_48hrs_icu/"
#import os
#os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# extract data recorded in ans prior to first 2hhrs of ICU stay
#df[df['icu_tdelta']<sec_48hrs].to_json(mypath_output+"microbioevents.json", orient = 'records')